In [ ]:
import numpy
import pandas
from pandas import read_csv
import sklearn
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.neighbors
import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import sklearn.linear_model
import sklearn.svm

babyData = read_csv("amazon_reviews_train_full.csv")

babyData = babyData.drop("review_headline",1)
babyData = babyData.drop("review_body",1)
babyData = babyData.drop("review_date",1)


babyData[["star_rating"]] = babyData[["star_rating"]].apply(pandas.to_numeric, downcast='integer')

print(babyData['star_rating'].value_counts())

babyData['five_or_not'] = numpy.where(babyData['star_rating'] == 5, True, False)

babyData = babyData.drop("star_rating",1)

babyData_num = babyData.select_dtypes(include=[numpy.number])

babyData_num.iloc[:,0:-1] = babyData_num.iloc[:,0:-1].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

babyData[babyData_num.columns] = babyData_num

babyData_dummies = pandas.get_dummies(babyData, prefix='', prefix_sep='', 
                            columns=babyData.select_dtypes(include=['object']).columns.tolist())

train, test = sklearn.model_selection.train_test_split(babyData_dummies)

train_y = train['five_or_not']
train_X = train.drop('five_or_not',1)
test_y = test['five_or_not']
test_X = test.drop('five_or_not',1)

print("KNN")
classifierKNN = sklearn.neighbors.KNeighborsClassifier()
classifierKNN.fit(train_X, train_y)
print(classifierKNN.score(test_X, test_y))

print("Naive Bayes")
classifierNB = sklearn.naive_bayes.GaussianNB() 
classifierNB.fit(train_X, train_y)
print(classifierNB.score(test_X, test_y))

print("Decision Tree")
classifierDT = sklearn.tree.DecisionTreeClassifier()
classifierDT.fit(train_X, train_y)
print(classifierDT.score(test_X, test_y))

print("Random Forest")
classifierRF = sklearn.ensemble.RandomForestClassifier()
classifierRF.fit(train_X, train_y)
print(classifierRF.score(test_X, test_y))

print("Logistic Regression")
classifierLR = sklearn.linear_model.LogisticRegression()
classifierLR.fit(train_X, train_y)
print(classifierLR.score(test_X, test_y))

#print("SVM")
#classifierSVM = sklearn.svm.SVC()
#classifierSVM.fit(train_X, train_y)
#print(classifierSVM.score(test_X, test_y))

In [ ]:
import numpy
import pandas
from pandas import read_csv
import sklearn
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.neighbors
import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import sklearn.linear_model
import sklearn.svm

babyData_train = read_csv("amazon_reviews_train_full.csv")

babyData_train = babyData_train.drop("review_id",1)
babyData_train = babyData_train.drop("review_headline",1)
babyData_train = babyData_train.drop("review_body",1)
babyData_train = babyData_train.drop("review_date",1)

babyData_train[["star_rating"]] = babyData_train[["star_rating"]].apply(pandas.to_numeric, downcast='integer')
babyData_train['five_stars'] = numpy.where(babyData_train['star_rating'] == 5, True, False)
babyData_train = babyData_train.drop("star_rating",1)

babyData_train_num = babyData_train.select_dtypes(include=[numpy.number])

myNormalizer = sklearn.preprocessing.Normalizer().fit(babyData_train_num.to_numpy())

babyData_train_num[:] = myNormalizer.transform(babyData_train_num.to_numpy())
babyData_train[babyData_train_num.columns] = babyData_train_num

babyData_train_cat = babyData_train.select_dtypes(include=[numpy.object])

babyData_train_cat_cols = babyData_train_cat.columns

myOneHotEncoder = sklearn.preprocessing.OneHotEncoder(handle_unknown = "ignore", sparse = False).fit(babyData_train_cat.to_numpy())

babyData_train_cat_dum = myOneHotEncoder.transform(babyData_train_cat.to_numpy())
babyData_dum_cats = myOneHotEncoder.get_feature_names()
babyData_train_cat_frame = pandas.DataFrame(babyData_train_cat_dum)
babyData_train_cat_frame.columns = babyData_dum_cats
babyData_train[babyData_dum_cats] = babyData_train_cat_frame
babyData_train = babyData_train.drop(babyData_train_cat_cols,1)

train_y = babyData_train['five_stars']
train_X = babyData_train.drop('five_stars',1)

babyData_predict = read_csv("amazon_reviews_test_features.csv")

babyData_predict = babyData_predict.drop("id",1)
babyData_predict = babyData_predict.drop("review_id",1)
babyData_predict = babyData_predict.drop("review_headline",1)
babyData_predict = babyData_predict.drop("review_body",1)
babyData_predict = babyData_predict.drop("review_date",1)

babyData_predict_num = babyData_predict.select_dtypes(include=[numpy.number])

babyData_predict_num[:] = myNormalizer.transform(babyData_predict_num.to_numpy())

babyData_predict[babyData_predict_num.columns] = babyData_predict_num

babyData_predict_cat = babyData_predict.select_dtypes(include=[numpy.object])

babyData_predict_cat_dum = myOneHotEncoder.transform(babyData_predict_cat.to_numpy())
babyData_dum_cats = myOneHotEncoder.get_feature_names()
babyData_predict_cat_frame = pandas.DataFrame(babyData_predict_cat_dum)
babyData_predict_cat_frame.columns = babyData_dum_cats
babyData_predict[babyData_dum_cats] = babyData_predict_cat_frame
babyData_predict = babyData_predict.drop(babyData_train_cat_cols,1)

classifierKNN = sklearn.neighbors.KNeighborsClassifier()
classifierKNN.fit(train_X, train_y)
myPrediction = pandas.DataFrame(classifierKNN.predict(babyData_predict))
myPrediction.columns = ["five_stars"]
myPrediction["ID"] = myPrediction.index+1

myPrediction.to_csv(path_or_buf="amazon_reviews_guesses.csv", index = False, index_label = False)
